In [ ]:
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.io import fits
from astropy.time import Time
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('apw-notebook')
import numpy as np
import h5py 

# Project
from comoving_rv.log import logger
from comoving_rv.longslit import SkippableImageFileCollection, voigt_polynomial
from comoving_rv.longslit.wavelength import fit_spec_line
from comoving_rv.velocity import bary_vel_corr, kitt_peak

In [ ]:
cache_file = '/Users/adrian/projects/gwb-spectra/data/mdm-spring-2017/processed/Halpha_centroids.h5'

In [ ]:
n_bg_coef = 2
Hα_slice = slice(655,720)

In [ ]:
for j in range(1,5+1):
    ic = SkippableImageFileCollection("/Users/adrian/projects/gwb-spectra/data/mdm-spring-2017/processed/n{}/".format(j),
                                      glob_pattr="1d_*")
    for fname in ic.files:
        hdulist = fits.open(path.join(ic.location, fname))
        hdu0 = hdulist[0]
        hdu1 = hdulist[1]
        try:
            group_id,obj_idx,*_ = hdu0.header['OBJECT'].split('-')
        except ValueError:
            print('Skipping object {}'.format(hdu0.header['OBJECT']))
            continue

        obj_idx = int(obj_idx)
        print("Group,idx: {} {}".format(group_id, obj_idx))

        # read RA, Dec from header
        obj_ra = coord.Angle(hdu0.header['RA'], unit=u.hourangle)
        obj_dec = coord.Angle(hdu0.header['DEC'], unit=u.degree)
        obj_jd = hdu0.header['JD']

        # read spectrum itself
        pix = hdu1.data['pix']
        flux = hdu1.data['source_flux']
        ivar = hdu1.data['source_ivar']

        with h5py.File(cache_file, 'a') as f:
            if group_id not in f:
                g = f.create_group(group_id)

                idx = g.create_dataset('idx', shape=(1,), maxshape=(None,), dtype='i4', chunks=True)
                ra = g.create_dataset('ra', shape=(1,), maxshape=(None,), dtype='f8', chunks=True)
                dec = g.create_dataset('dec', shape=(1,), maxshape=(None,), dtype='f8', chunks=True)
                jd = g.create_dataset('JD', shape=(1,), maxshape=(None,), dtype='f8', chunks=True)

                # line centroid
                amp = g.create_dataset('amp', shape=(1,), maxshape=(None,), dtype='f8', chunks=True)
                x0 = g.create_dataset('x_0', shape=(1,), maxshape=(None,), dtype='f8', chunks=True)
                std_G = g.create_dataset('std_G', shape=(1,), maxshape=(None,), dtype='f8', chunks=True)
                fwhm_L = g.create_dataset('fwhm_L', shape=(1,), maxshape=(None,), dtype='f8', chunks=True)
                bg_coef = g.create_dataset('bg_coef', shape=(1,n_bg_coef), maxshape=(None,n_bg_coef), 
                                           dtype='f8', chunks=True)
                Cov = g.create_dataset('Cov', shape=(1,4+n_bg_coef,4+n_bg_coef), 
                                       maxshape=(None,4+n_bg_coef,4+n_bg_coef), dtype='f8', chunks=True)

            else:
                g = f[group_id]
                idx = g['idx'][:]

                if obj_idx in idx:
                    continue

                # obj idx not in file, but group is - need to resize arrays
                n_group = len(idx)
                for key in g.keys():
                    shape = list(g[key].shape)
                    shape[0] += 1
                    g[key].resize(tuple(shape))

            for sign in [-1, 1]:
                try:
                    par,cov = fit_spec_line(x=pix[Hα_slice], flux=flux[Hα_slice], 
                                            flux_ivar=ivar[Hα_slice], 
                                            absorp_emiss=sign, return_cov=True, target_x=687)
                    break
                except (ValueError,RuntimeError):
                    print("Failed to fit for sign={}".format(sign))

            else:
                print("Totally failed")
                continue

            g['idx'][-1] = int(obj_idx)
            g['ra'][-1] = obj_ra.degree
            g['dec'][-1] = obj_dec.degree
            g['JD'][-1] = obj_jd

            for key in par.keys():
                g[key][-1] = par[key]

            if cov is None:
                g['Cov'][-1] = np.full((4+n_bg_coef,4+n_bg_coef), np.nan)
            else:
                g['Cov'][-1] = cov

In [ ]:
plt.plot(pix[Hα_slice], flux[Hα_slice])

In [ ]:
all_dv = []
all_dv_err = []
all_dvbary = []
all_sep = []
with h5py.File(cache_file, 'r') as f:
    for group_id in f.keys():
        n_members = len(f[group_id]['idx'])
        if n_members != 2:
            continue
        
        x0s = f[group_id]['x_0'][:]
        dv = 2*(x0s[0]-x0s[1]) / 6562.8 * 300000. * u.km/u.s
        all_dv.append(dv)
        
        dx0s = np.sqrt(f[group_id]['Cov'][:,1,1])
        dv_err = 2*dx0s / 6562.8 * 300000. * u.km/u.s
        all_dv_err.append(np.sqrt(dv_err[0]**2 + dv_err[1]**2))
        
        # angular separation
        cs = coord.ICRS(ra=f[group_id]['ra'][:]*u.degree,
                        dec=f[group_id]['dec'][:]*u.degree)
        sep = cs[0].separation(cs[1])
        all_sep.append(sep.to(u.degree).value*u.degree)
        
        # barycentric correction
        t = Time(f[group_id]['JD'][:], format='jd', scale='utc')
        vbary = bary_vel_corr(t, cs, location=kitt_peak)
        all_dvbary.append(vbary[0]-vbary[1])
        
all_dv = u.Quantity(all_dv)
all_dv_err = u.Quantity(all_dv_err)
all_sep = u.Quantity(all_sep)
all_dvbary = u.Quantity(all_dvbary)

In [ ]:
all_dv_corr = all_dv - all_dvbary

all_dv_corr = all_dv_corr[np.isfinite(all_dv_err)]
all_sep = all_sep[np.isfinite(all_dv_err)]
all_dv_err = all_dv_err[np.isfinite(all_dv_err)]

In [ ]:
(np.abs(all_dv_corr) < 25*u.km/u.s).sum()

In [ ]:
plt.hist(all_dv_corr.value[np.abs(all_dv_corr.value) < 100.], bins='auto');

In [ ]:
fig,axes = plt.subplots(2, 1, figsize=(8,8),sharex=True)

axes[0].errorbar(all_sep.value, np.abs(all_dv_corr.value), all_dv_err.value,
                 marker='.', alpha=0.5, linestyle='none', markeredgewidth=0)
axes[0].set_xscale('log')
axes[0].set_xlim(1E-3, 20)
axes[0].set_ylim(-1, 10)
axes[0].set_ylabel(r'$\Delta {\rm RV}\,[{\rm km}\,{\rm s}^{-1}]$')

axes[1].errorbar(all_sep.value, np.abs(all_dv_corr.value), all_dv_err.value,
                 marker='.', alpha=0.5, linestyle='none', markeredgewidth=0)
axes[1].set_ylim(-1, 50)
axes[1].set_xlabel('ang. sep. [deg]')
axes[1].set_ylabel(r'$\Delta {\rm RV}\,[{\rm km}\,{\rm s}^{-1}]$')

fig.tight_layout()
fig.savefig('/Users/adrian/Downloads/dv_relative.png')

## Group 10

In [ ]:
with h5py.File(cache_file, 'r') as f:
    x0s = f['10/x_0'][:]
    dx = x0s - np.median(x0s)
    group10_dv = 2*dx / 6562.8 * 300000. * u.km/u.s

In [ ]:
plt.hist(group10_dv.value, bins=np.linspace(-50, 50, 16));

In [ ]:
np.std(group10_dv.value[(group10_dv.value > -50) & (group10_dv.value < 50)])

TODO: But it covers some large area on the sky, right?

---

In [ ]:
t = Time(obj_jd, format='jd', scale='utc')
sc1 = coord.SkyCoord(ra=obj_ra, dec=obj_dec)
sc2 = coord.SkyCoord(ra=obj_ra+2.*u.deg, dec=obj_dec+-5*u.deg)

In [ ]:
ra_grid = np.linspace(0, 360, 128)
sc1 = coord.SkyCoord(ra=ra_grid*u.deg, dec=np.full_like(ra_grid, -23)*u.deg)

In [ ]:
vel = bary_vel_corr(t, sc1, location=kitt_peak) - bary_vel_corr(t, sc2, location=kitt_peak)

In [ ]:
plt.plot(vel)